In [1]:

import os, sys, math, time
import numpy as np
import numpy.linalg as la
import plotly.graph_objects as go
import plotly.express as ex
from plotly.subplots import make_subplots
import pandas as pd

import json as js
import _pickle as pickle
import bz2
import ray

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from collections import OrderedDict

from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import shutil
import tempfile
from ray.tune import CLIReporter, JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler, PopulationBasedTraining
from ray.tune.integration.pytorch_lightning import TuneReportCallback, \
    TuneReportCheckpointCallback

import pytorch_lightning as pl
from pytorch_lightning.utilities.cloud_io import load as pl_load
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping

from cytoolz import sliding_window
sys.path.append("../")
import func

In [2]:
# Test torch lightning + ray tune
ray.shutdown()
# Prepare train data
DATA_PATH = "/home/nuoc/Documents/MEX/data"
MODEL_PATH = "/home/nuoc/Documents/MEX/models"
RESULTS_PATH = "/home/nuoc/Documents/MEX/results"

In [3]:
class MLP(pl.LightningModule):
    def __init__(self, config:dict=None, dimensions:list=None, loss_fn=None, extra_feature_len:int=None,
                 dataset=None, train_set=None, val_set=None, test_set=None,
                 keep_prob:float=.2, name:str="model", load=False,
                 single_module:int=0):

        super(MLP, self).__init__()
        self.name = name
        self.dimensions = dimensions
        self.keep_prob = keep_prob
        self.single_module = single_module
        self.extra_feature_len = extra_feature_len

        if load:
            self.build()
        else:
            self.k = config["k"]
            self.learning_rate = config["lr"]
            self.dimensions = dimensions + [config["hidden_dim"]] + [self.k]
            self.batch_size = config["batch_size"]
            self.loss_fn = config["loss_fn"]
            self.keep_prob = keep_prob

            self.dataset = dataset

            self.train_set = train_set
            self.val_set = val_set
            self.test_set = test_set
            self.best_val_loss = np.inf

            self.build()
            self.encoder.apply(self.init_params)
            self.decoder.apply(self.init_params)

    def build(self):
        layer_sizes = list(sliding_window(2, self.dimensions))
        if self.single_module == -1 or self.single_module == 0:
            layers = []

            for i, size in enumerate(layer_sizes):
                layers.append(("fc"+str(i), nn.Linear(size[0], size[1])))
                if i < len(self.dimensions)-2:
                    layers.append(("act"+str(i), nn.ELU()))
                    layers.append(("drop"+str(i+1), nn.Dropout(self.keep_prob)))
            self.encoder = nn.Sequential(OrderedDict(layers))
        else:
            self.encoder = nn.Sequential()

        if self.single_module == 0 or self.single_module == 1:
            layers = []
            layer_sizes[-1] = (layer_sizes[-1][0], layer_sizes[-1][1] + self.extra_feature_len)
            for i, size in enumerate(layer_sizes[-1::-1]):
                layers.append(("fc"+str(i), nn.Linear(size[1], size[0])))
                if i < len(self.dimensions)-2:
                    layers.append(("act"+str(i), nn.ELU()))
                    layers.append(("drop"+str(i+1), nn.Dropout(self.keep_prob)))
            self.decoder = nn.Sequential(OrderedDict(layers))
        else:
            self.decoder = nn.Sequential()

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        _x, label = x[:, :-self.extra_feature_len], x[:, -self.extra_feature_len:]
        h = self.encoder(_x)
        hr = torch.cat((h, label), dim=1)
        return self.decoder(hr)

    def training_step(self, batch, batch_idx):
        x, y = batch
        prediction = self(x)
        loss = self.loss_fn(prediction, y)

        self.log("ptl/train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch

        prediction = self(x)
        loss = self.loss_fn(prediction, y)

        self.log('ptl/val_loss', loss, prog_bar=True)
        return {"val_loss":loss}

    def test_step(self, batch, batch_idx):
        x, y = batch

        prediction = self(x)
        loss = self.loss_fn(prediction, y)

        self.log('ptl/test_loss', loss, prog_bar=True)
        return {"val_loss":loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        self.log("avg_val_loss", avg_loss)
        if avg_loss < self.best_val_loss:
            self.best_val_loss = avg_loss
            self.save_checkpoint()

    def save_checkpoint(self, checkpoint_dir=MODEL_PATH):
        path = os.path.join(checkpoint_dir, self.name)
        model = {"k":self.k, "dimensions":self.dimensions,"keep_prob":self.keep_prob, "name":self.name,
                 "extra_feature_len" : self.extra_feature_len,
                 "encoder":self.encoder.state_dict(),
                 "decoder":self.decoder.state_dict()}
        if not os.path.exists(path):
            os.mkdir(path)
        with bz2.BZ2File(os.path.join(path,
                                      str(self.best_val_loss.cpu().numpy())+"."+str(self.k)+".pbz2"), "w") as f:
            pickle.dump(model, f)

    @staticmethod
    def load_checkpoint(filename):
        # return torch.load(os.path.join(checkpoint_dir,filename))
        with bz2.BZ2File(filename, "rb") as f:
            obj = pickle.load(f)
        model = MLP(name=obj["name"], dimensions=obj["dimensions"], extra_feature_len=obj["extra_feature_len"], load=True)
        model.encoder.load_state_dict(obj["encoder"])
        model.decoder.load_state_dict(obj["decoder"])
        return model
        # self.encoder.load_state_dict(obj["encoder"])
        # self.decoder.load_state_dict(obj["decoder"])
        # self.best_val_loss = obj["val_loss"]

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        return optimizer

    def setup_data(self, split_ratio, N):
        self.n_train_samples= int(split_ratio[0]*N)
        self.n_val_samples= int(split_ratio[1] * N)
        self.n_test_samples= int(N-self.n_train_samples-self.n_val_samples)
        self.train_set, self.val_set, self.test_set = random_split(self.dataset,
                                                                   [self.n_train_samples,
                                                                    self.n_val_samples,
                                                                    self.n_test_samples])

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, pin_memory=True)

    @staticmethod
    def init_params(m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(.01)

In [4]:
def train_tune(config, dimensions:list,  loss_fn=None, extra_feature_len:int=0,
                 train_set=None, val_set=None, test_set=None,
                 keep_prob:float=.2, num_epochs=300, num_cpus=24, num_gpus=1, model_name="model"):

    model = MLP(config=config, dimensions=dimensions, loss_fn=loss_fn,
                train_set=train_set, val_set=val_set, test_set=test_set,
                keep_prob=keep_prob, name=model_name,extra_feature_len=extra_feature_len)
    trainer = pl.Trainer(
        max_epochs=num_epochs,
        gpus=num_gpus,
        logger=TensorBoardLogger(save_dir="logs/", name=model_name, version="0.0"),
        progress_bar_refresh_rate=20,
        callbacks=[
            TuneReportCallback({"loss":"avg_val_loss",}, on="validation_end"),
            EarlyStopping(monitor="avg_val_loss")
        ],
        precision=16,
    )
    trainer.fit(model)

def normalise(x:torch.Tensor):
    std = torch.std(x)
    std[std==0] = 1
    return (x - torch.mean(x)) / std


In [5]:
def prepare_data(datasets:list, featureList:list, extra_feature_len:int=0,
                 train_ratio:float=0.8, val_ratio:float=0.2, test_size:int=100, SEED:int=2021):
   # process data
    data = [func.processData(d, featureList) for d in datasets]
    input_data = [np.vstack(d) for d in data]
    x_tensors = [func.normaliseT(torch.from_numpy(x).float()) for x in input_data]
    y_tensors = [torch.from_numpy(x[:, :-extra_feature_len]).float() for x in input_data]

    # prepare datasets
    test_sets = [(x_tensor[-test_size:], y_tensor[-test_size:]) for x_tensor, y_tensor in zip(x_tensors, y_tensors)]
    x_training = torch.vstack([x_tensor[:-test_size] for x_tensor in x_tensors])
    y_training = torch.vstack([y_tensor[:-test_size] for y_tensor in y_tensors])
    dataset = TensorDataset(x_training, y_training)
    N = len(x_training)

    train_ratio = int(train_ratio*N)
    val_ratio = int(val_ratio*N)
    print("Train: ", train_ratio, ", Validation: ", val_ratio)
    train_set, val_set = random_split(dataset, [train_ratio, val_ratio], generator=torch.Generator().manual_seed(SEED))

    return train_set, val_set, test_sets

def train(train_set:Dataset, val_set:Dataset,config:dict,
          extra_feature_len:int=-1, EPOCHS:int=300, hidden_dim:int=256,
          loss_fn=torch.nn.functional.mse_loss, keep_prob=0.2,
          n_gpu=1, n_samples=20, model_name="model",
          ):

    # define hyperparameters
    input_dim = train_set[0][0].size()[-1] - extra_feature_len
    # output_dim = input_dim

    dimensions = [input_dim]

    scheduler = ASHAScheduler(max_t = EPOCHS, grace_period=1, reduction_factor=2)
    reporter = CLIReporter(
        parameter_columns=["k", "lr", "batch_size", "loss_fn", "hidden_dim"],
        metric_columns=["loss", "training_iteration"],
        max_error_rows=5,
        max_progress_rows=5,
        max_report_frequency=10)
    analysis = tune.run(
        tune.with_parameters(
            train_tune,
            dimensions = dimensions,
            loss_fn = loss_fn,
            train_set = train_set, val_set = val_set,
            extra_feature_len=extra_feature_len,
            keep_prob = keep_prob,
            num_epochs = EPOCHS,
            num_gpus=n_gpu,
            model_name=model_name
        ),
        resources_per_trial= {"cpu":1, "gpu":n_gpu},
        metric="loss",
        mode="min",
        config=config,
        num_samples=n_samples,
        scheduler=scheduler,
        progress_reporter=reporter,
        name="MLP-MLP",
        verbose=False,
        checkpoint_freq=0,
        keep_checkpoints_num=1,
        checkpoint_score_attr="loss",
        checkpoint_at_end=True
    )

    print("-"*70)
    print("Done")
    print("Best hyperparameters found were: ", analysis.best_config)
    print("Best achieved loss was: ", analysis.best_result)
    print("-"*70)

def clean_checkpoints(num_keep=3, path="../../models"):
    saved_checkpoints = []
    for dir, dname, files in os.walk(path):
        for fname in files:
            fname = fname.split(".")
            saved_checkpoints.append(fname)
        print("Num checkpoints in {}: {}".format(dir, len(saved_checkpoints)))

        saved_checkpoints.sort(key = lambda x: x[0]+x[1])
        for filename in saved_checkpoints[num_keep:]:
            os.remove(os.path.join(dir,".".join(filename)))
        break
    return os.path.join(path, ".".join(saved_checkpoints[0]))

def test(model:torch.nn.Module, test_sets:list, loss_fn, set_names=list,
         save=True, path="../../results"):
    # Intra test performance
    df = {}
    if set_names is None: set_names = np.arange(len(test_sets))
    for i,t1 in enumerate(test_sets):
        for j, t2 in enumerate(test_sets):
            x = t1[0]
            y = t2[1]
            pred = model(x)
            loss = loss_fn(x, y)
            df["{}-{}".format(set_names[i], set_names[j])] = [loss.cpu().numpy()]
            print("Test encoding {} to {}, MSE={:.2f}".format(set_names[i], set_names[j], loss))
    if save:
        if not os.path.exists(path): os.mkdir(path)
        pd.DataFrame(df).to_csv(os.path.join(path, "tests.csv"))

# Testing pos, rotMat, velocity

In [6]:
# Prepare train data
DATA_PATH = "/home/nuoc/Documents/MEX/data"
MODEL_PATH = "/home/nuoc/Documents/MEX/models"
RESULTS_PATH = "/home/nuoc/Documents/MEX/results"

def train_single_model(datapaths:list, featureList:list,config:dict=None, extra_feature_len:int=0,
                       n_samples:int=30, model_name:str="model", loss_fn=nn.functional.mse_loss,
                       dataset_names:list=None):
    # load data
    datasets = [func.load(os.path.join(DATA_PATH,path)) for path in datapaths]
    train_set, val_set, test_set = prepare_data(datasets, featureList, extra_feature_len=extra_feature_len)
    train(train_set=train_set, val_set=val_set, config=config, loss_fn=loss_fn, extra_feature_len=extra_feature_len,
          n_samples=n_samples, model_name=model_name)

    best_model = clean_checkpoints(path=os.path.join(MODEL_PATH, model_name))
    best_model = MLP.load_checkpoint(best_model)
    test(best_model, test_set, loss_fn=loss_fn, set_names=dataset_names, path=os.path.join(RESULTS_PATH, model_name))




In [7]:
datapaths = ["LOCO_R1-default-locomotion.pbz2",
             "LOCO_R1-default-locomotion-small.pbz2",
             "LOCO_R1-default-locomotion-large.pbz2"]
featureList = ["pos", "rotMat", "velocity", "isLeft", "chainPos", "geoDistanceNormalised"]

extra_feature_len = 21 * 3

In [8]:
config = {
    "k":tune.randint(2, 256),
    "lr": tune.loguniform(1e-3, 1e-7),
    "batch_size":tune.choice([5, 15, 30, 60]),
    "hidden_dim": tune.choice([32, 64, 128, 256]),
    "loss_fn" : tune.choice([nn.functional.mse_loss, nn.functional.smooth_l1_loss])
}

train_single_model(datapaths=datapaths, featureList=featureList, config=config, extra_feature_len=extra_feature_len,
                   model_name="MLP-MLP_extra", dataset_names=["default", "small", "large"])


2021-04-01 12:11:00,305	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-04-01 12:11:04,422	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-04-01 12:11:08,278	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-04-01 12:11:11,859	INFO ray_trial_executor.py:197 -- Initializing Ray automatically.For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run`.
2021-04-01 12:11:12,255	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-04-01 12:11:13,590	WARNING function_runner.py:540 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
(pid=719346) GPU available: True, used: True
(pid=719346) TPU available: None, using: 0 TPU cores
(pid=719346) Using native 16bit precision.
(pid=719346) LOCA

Train:  3216 , Validation:  804
Epoch 0:  88%|████████▊ | 60/68 [00:00<00:00, 463.78it/s, loss=0.494, v_num=0.0, ptl/val_loss=0.439]
Validating: 0it [00:00, ?it/s]
Epoch 0:  88%|████████▊ | 60/68 [00:00<00:00, 463.78it/s, loss=0.494, v_num=0.0, ptl/val_loss=0.439]
Validating: 0it [00:00, ?it/s]
Epoch 0:  88%|████████▊ | 60/68 [00:00<00:00, 463.78it/s, loss=0.494, v_num=0.0, ptl/val_loss=0.439]
Validating: 0it [00:00, ?it/s]
Epoch 0:  88%|████████▊ | 60/68 [00:00<00:00, 463.78it/s, loss=0.494, v_num=0.0, ptl/val_loss=0.439]
Validating: 0it [00:00, ?it/s]
Epoch 1:  88%|████████▊ | 60/68 [00:00<00:00, 455.85it/s, loss=0.469, v_num=0.0, ptl/val_loss=0.423]
Validating: 0it [00:00, ?it/s]
Epoch 1:  88%|████████▊ | 60/68 [00:00<00:00, 455.85it/s, loss=0.469, v_num=0.0, ptl/val_loss=0.423]
Validating: 0it [00:00, ?it/s]
Epoch 1:  88%|████████▊ | 60/68 [00:00<00:00, 455.85it/s, loss=0.469, v_num=0.0, ptl/val_loss=0.423]
Validating: 0it [00:00, ?it/s]
Epoch 1:  88%|████████▊ | 60/68 [00:00<00:00

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'